# *Simple Video Re-Encoder*
Made with `FFmpeg`

In [ ]:
#@title # <font color="lime"><b>❮</b></font> Upload and process video
#@markdown ##### ...after filling the forms below ⬇️
#@markdown ---

#@markdown ## `File 📝`
URL = "" #@param {type:"string"}
Keep_Metadata = False #@param {type:"boolean"}
Output_Preview_Volume = 7.5 #@param {type:"slider", min:0.1, max:50, step:0.1}
#@markdown ## `Audio 🔊`
Channels = "Copy" #@param ["Mono", "Stereo", "Copy"]
Bitrate_kbps = "320" #@param ['320', '256', '224', '192', '160', '128', '96']
Samplerate_Hz = "Copy" #@param ["48 000", "44 100", "32 000", "Copy"]
#@markdown ## `Video 🎥`
Bitrate_mbps = "Copy" #@param ["10", "9", "8", "7", "6", "5", "4", "3", "2", "Copy"]
Output_Format_and_Codec = "MP4 (x264)" #@param ["MP4 (x264)", "WEBM (VP9)"]
#@markdown ---
#@markdown ## Notes ⚠️
#@markdown - All previous files are removed on each new upload.
#@markdown - `Copy` statement means inheritance of original attribute of uploaded file.

URL = URL.split("?")[0].split("#")[0]
Keep_Metadata = "-map 0 -map_metadata 0:s:0 " if not Keep_Metadata else ""
Output_Preview_Volume /= 100
Audio_Channels = ("-ac " + ("1" if Channels == "Mono" else "2")) if not Channels == "Copy" else ""
Audio_Bitrate = "-b:a " + str(Bitrate_kbps) + "k"
Samplerate = ("-ar " + Samplerate_Hz.replace(" ", "")) if not Samplerate_Hz == "Copy" else ""
Video_Bitrate = ("-b:v " + str(int(Bitrate_mbps) * 1000) + "k") if not Bitrate_mbps == "Copy" else ""
Output_Format_and_Codec = [Param.lower() for Param in Output_Format_and_Codec.split(" ")]
Output_Format = Output_Format_and_Codec[0]
Video_Codec = "-c:v " + ("lib" if Output_Format == "mp4" else "") + Output_Format_and_Codec[1][1:-1]

#---#

from re import sub
from time import time
from shutil import rmtree
from os import *; del open
from datetime import timedelta
from google.colab import files
from base64 import b64encode as b64
from requests import get, exceptions
from IPython.display import display, HTML

#---#

__author__				= "kubinka0505"
__copyright__		= __author__
__credits__				= __author__
__version__				= "1.0"
__date__					= "08.08.2021"
__status__				= "Mature"
__license__				= "GPL V3"

#---#

chdir("/content/")
CWD = getcwd()
chdir("..")

for Command in [rmtree, mkdir, chdir]:
	Command(CWD)

#---#

def File_Size(Size: bytes):
	for Unit in ['B', 'KB', 'MB', 'GB', 'TB', 'PB']:
		if Size < 1024. or Unit == "PB": 
			break
		Size /= 1024.
	return "{0} {1}".format(round(Size, 2), Unit)

class Styles():
	"""Formatted Prints."""
	LightRed	= "\33[31m"
	Blue			= "\33[34m"
	Red				= "\33[91m"
	Green			= "\33[92m"
	Yellow		= "\33[93m"
	Cyan			= "\33[96m"
	Bold			= "\33[1m"
	Underline	= "\33[4m"
	Reset			= "\33[0m"

Allowed_Formats = ["AVI", "MKV", "MOV", "MP4", "WEBM"]

#---#

if not URL:
	try:
		File = [Key for Key in files.upload().keys()][0]
		print()
	except (TypeError, IndexError, KeyboardInterrupt):
		raise SystemExit("File upload aborted.")
else:
	print("Uploading from {0}URL{1}...".format(
		Styles.Yellow, Styles.Reset
		)
	)
	
	File = URL.split("/")[-1]
	with open(File, "wb") as File_:
		try:
			Request = get(URL)
		except exceptions.MissingSchema:
				raise SystemExit("Invalid URL.")
		if Request.status_code == 200:
			File_.write(Request.content)
		else:
			raise SystemExit("Cannot import URL - {2}{0}{3} ({2}{1}{3})".format(
				Request.status_code, Request.reason, Styles.Red, Styles.Reset)
			)
try:
	In = path.abspath(File)
	In_NEW = sub("[^\w\-_\. ]", "_", In.split("/")[-1]).replace(" ", "_")
	rename(In, In_NEW)
	In = path.abspath(In_NEW)

	Out = In.split(".")[0] + "_new." + Output_Format

	if not In.split(".")[-1].upper() in Allowed_Formats:
		chmod(In, 0o777)
		remove(In)
		print("{2}{0}{3}: Uploaded file is not a video!\nAllowed formats: {1}".format(
			TypeError.__class__.__name__ + "{1}, {0}".join(Allowed_Formats).format(
				Styles.Red, Styles.Reset
			),
			Styles.Red, Styles.Reset
			)
		)

	#---#

	environ["IN"] = In
	environ["OUT"] = Out
	environ["AC"] = Audio_Channels
	environ["BA"] = Audio_Bitrate
	environ["AR"] = Samplerate
	environ["BV"] = Video_Bitrate
	environ["CV"] = Video_Codec
	environ["MD"] = Keep_Metadata

	print("Re-encoding...")
	__STA_TIME = time()
	!ffmpeg -i "$IN" $BV $CV $AC $AR $BA $MD "$OUT" -y -hide_banner -loglevel warning

	#---#

	Size_IN = path.getsize(In)
	Size_OUT = path.getsize(Out)
	Percentage = round(
			100 - (
				float(str(Size_OUT).split(" ")[0]) / float(str(Size_IN).split(" ")[0])
			) * 100, 3
		) if not Size_IN == Size_OUT else "0.00"
	print("""
	Size changed by:\t{4}{2}%{8}
	Base file size:\t\t{5}{0}{8}
	New file size:\t\t{6}{1}{8}

	Work time: {7}{3}{8}\n""".replace("\n	", "\n").format(
		File_Size(Size_IN), File_Size(Size_OUT), Percentage,
		str(timedelta(seconds = -__STA_TIME + time()))[2:-3],
		Styles.Cyan, Styles.LightRed, Styles.Green, Styles.Blue, Styles.Reset
		)
	)

	B64 = "data:video/{0};base64,".format(Output_Format) + b64(open(Out, "rb").read()).decode()

	display(
		HTML(
			"""<hr><br><video id="Video" width=400 autoplay controls src="{0}" type="video/{1}">
			</video><script>document.getElementById("Video").volume = {2};</script>""".format(
				B64, Output_Format, Output_Preview_Volume)
			)
		)

	print()
	files.download(Out)
except Exception as Error:
	print(Error.__class__.__name__ + ":  " + Error)